In [1]:
#Import all the packages
import pandas as pd
import numpy as np
from collections import OrderedDict
from neo4j import GraphDatabase
import json
import itertools

In [2]:
#Function to construct neo4j graph commands
def neo4j_commands(file_path):
    with open(file_path, mode="r") as d_f:
        file_content = d_f.readline()
        RequirementChain = json.loads(file_content)
    transaction_execution_commands = []
    pos = 0
    total_objects = []
    neo4j_create_statement = "CREATE "
    for i in RequirementChain:
        #Creating blocks and connections between them
        neo4j_create_statement = neo4j_create_statement + "(block" +  str(pos) 
        neo4j_create_statement = neo4j_create_statement +":block{name: \"block" +  str(pos) + "\","
        for j, k in enumerate(i):
            if k == "previous_hash" or k == "hash":
                neo4j_create_statement = neo4j_create_statement + str(k) +': \"' + str(i[k]) + "\","
        neo4j_create_statement = neo4j_create_statement[:-1]
        neo4j_create_statement = neo4j_create_statement + "}),"
        
        #defining relations between blocks
        if pos >0:
            pre_pos = pos-1
            neo4j_create_statement = neo4j_create_statement + "(block" + str(pre_pos) 
            neo4j_create_statement = neo4j_create_statement + ") -[r" + str(pre_pos) + ":CONNECTS]-> " 
            neo4j_create_statement = neo4j_create_statement + "(block" + str(pos) +")," 

        #Creating artifacts and connecting them to blocks
        #neo4j_create_statement = neo4j_create_statement + "(artifact" +  str(pos) +":artifact{"
        neo4j_create_statement = neo4j_create_statement + "(art" + str(i["hash"]) +":artifact{"
        for j, k in enumerate(i):
            if k == "name":
                neo4j_create_statement = neo4j_create_statement + str(k) +': \"' 
                neo4j_create_statement = neo4j_create_statement + str(i[k][0]) + "\"," + "Objects" 
                neo4j_create_statement = neo4j_create_statement + ': \"' 
                neo4j_create_statement = neo4j_create_statement + str(np.array(list(i["artifact_data"].values()))[:,0]) 
                neo4j_create_statement = neo4j_create_statement + "\","
        neo4j_create_statement = neo4j_create_statement[:-1]
        neo4j_create_statement = neo4j_create_statement + "}),"
        
        #defining relations between block and artifact name
        neo4j_create_statement = neo4j_create_statement + "(block" + str(pos) 
        neo4j_create_statement = neo4j_create_statement + ") -[a" + str(pos) + ":DOCUMENTS]-> " 
        neo4j_create_statement = neo4j_create_statement + "(art" + str(i["hash"])+"),"
        
        #Creating stakeholders and connecting them to blocks
        neo4j_create_statement = neo4j_create_statement + "(stakeholders" +  str(pos) +":stakeholder{"
        for j, k in enumerate(i):
            if k == "authors":
                neo4j_create_statement = neo4j_create_statement + str(k) +': \"' 
                for sk in i[k]:
                    neo4j_create_statement = neo4j_create_statement + str(sk) + ","
                neo4j_create_statement = neo4j_create_statement[:-1] + "\","
        neo4j_create_statement = neo4j_create_statement[:-1]
        neo4j_create_statement = neo4j_create_statement + "}),"
        
        #defining relations between block and creators
        neo4j_create_statement = neo4j_create_statement + "(stakeholders" + str(pos)
        #neo4j_create_statement = neo4j_create_statement + "(block" + str(pos) 
        neo4j_create_statement = neo4j_create_statement + ") -[skb" + str(pos) + ":CREATES]-> " 
        neo4j_create_statement = neo4j_create_statement + "(block" + str(pos)  +"),"
        
        #defining relations between artifact and artifact
        jpos = 1
        for j in i["artifact_trace"]:
            if pos>0:
                link_temp = str(j[1]).split(" ")
                link_true = ""
                for lk in link_temp:
                    link_true = link_true + str(lk) + "_"
                link_true2 = link_true[:-1]
                if str(j[1]) == "EVOLVED_TO" or str(j[1]) == "SATISFIES":
                    neo4j_create_statement = neo4j_create_statement + "(art" + str(j[0])
                    neo4j_create_statement = neo4j_create_statement + ") -[aa" + str(pos) + str(jpos) 
                    #neo4j_create_statement = neo4j_create_statement + ":" + str(j[1]) + "]-> " 
                    neo4j_create_statement = neo4j_create_statement + ":" + link_true2 + "]-> "
                    neo4j_create_statement = neo4j_create_statement +  "(art" + str(i["hash"]) +"),"
                else: 
                    neo4j_create_statement = neo4j_create_statement + "(art" + str(i["hash"])
                    neo4j_create_statement = neo4j_create_statement + ") -[aa" + str(pos) + str(jpos) 
                    #neo4j_create_statement = neo4j_create_statement + ":" + str(j[1]) + "]-> " 
                    neo4j_create_statement = neo4j_create_statement + ":" + link_true2 + "]-> "
                    neo4j_create_statement = neo4j_create_statement +  "(art" + str(j[0]) +"),"
                jpos = jpos+1
        #defining relations between block and artifact name
        neo4j_create_statement = neo4j_create_statement + "(stakeholders" + str(pos)
        #neo4j_create_statement = neo4j_create_statement + "(art" + str(i["hash"]) 
        neo4j_create_statement = neo4j_create_statement + ") -[ska" + str(pos) + ":HAS_ROLE_IN]-> " 
        neo4j_create_statement = neo4j_create_statement + "(art" + str(i["hash"])  +"),"
        
        #Creating stakeholders and connecting them to blocks
        Object=[]
        for j, k in enumerate(i):
            if k == "artifact_data":
                for s, p in enumerate(i[k]):
                    Object.append([i[k][p][1], i[k][p][0], i["hash"], i[k][p][4]])
                    #total_objects.append(i[k][p][1])
        Object.sort
        Object2 = list(k for k,_ in itertools.groupby(Object))
        #temp_objects = []
        for oi in range(len(Object2)):
            if Object2[oi][0] not in total_objects:
                neo4j_create_statement = neo4j_create_statement + "(obj" + str(Object2[oi][0]) + ":Object{name:"
                neo4j_create_statement = neo4j_create_statement +' \"' + str(Object2[oi][1]) 
                neo4j_create_statement = neo4j_create_statement+ '\"' + ",public_key:" 
                neo4j_create_statement = neo4j_create_statement+ ' \"' + str(Object2[oi][3]) + "\"}),"
                #temp_objects.append(Object2[oi][0])
            neo4j_create_statement = neo4j_create_statement + "(art" + str(Object2[oi][2]) 
            neo4j_create_statement = neo4j_create_statement + ") -[ar" + str(pos) + "_" + str(oi) 
            neo4j_create_statement = neo4j_create_statement + ":CONTAINS_OBJECT]-> " 
            neo4j_create_statement = neo4j_create_statement + "(obj" + str(Object2[oi][0]) +"),"

        for j, k in enumerate(i):
            if k == "artifact_data":
                for s, p in enumerate(i[k]):
                    total_objects.append(i[k][p][1])


        for j, k in enumerate(i):
            if k == "artifact_data":
                for s, p in enumerate(i[k]):
                    #p_temp = i[k][p][1]
                    link_temp = str(i[k][p][3]).split(" ")
                    link_true = ""
                    for lk in link_temp:
                        link_true = link_true + str(lk) + "_"
                    link_true2 = link_true[:-1]
                    if pos >0:
                        if str(i[k][p][3]) == "EVOLVED_TO" or str(i[k][p][3]) == "SATISFIES":
                            neo4j_create_statement = neo4j_create_statement + "(obj" + str(i[k][p][2])
                            neo4j_create_statement = neo4j_create_statement + ") -[re" + str(pos) +"_" 
                            #neo4j_create_statement = neo4j_create_statement + str(s) + ":" + str(i[k][p][3]) 
                            neo4j_create_statement = neo4j_create_statement + str(s) + ":" + str(link_true2)
                            neo4j_create_statement = neo4j_create_statement + "]-> " + "(obj" + str(i[k][p][1]) +"),"
                        else:
                            neo4j_create_statement = neo4j_create_statement + "(obj" + str(i[k][p][1])
                            neo4j_create_statement = neo4j_create_statement + ") -[re" + str(pos) +"_" 
                            #neo4j_create_statement = neo4j_create_statement + str(s) + ":" + str(i[k][p][3]) 
                            neo4j_create_statement = neo4j_create_statement + str(s) + ":" + str(link_true2)
                            neo4j_create_statement = neo4j_create_statement + "]-> " + "(obj" + str(i[k][p][2]) +"),"
        pos = pos + 1
    transaction_execution_commands.append(neo4j_create_statement[:-1])
        #neo4j_create_statement = []
    return(transaction_execution_commands)
#Function to execute the neo4j graph commands
def execute_requirementblock(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "123456"))
    #data_base_connection = GraphDatabase.driver(uri = "http://127.0.0.10:5000/", auth=("neo4j", "password"))
    session = data_base_connection.session()
    for i in transaction_execution_commands:
        session.run(i)

In [12]:
#Read the requirement chain from the file
path1 = 'C:\\Users\\mohan\\Desktop\\Experiement_blockchain1\\'
path2 = "blockchain_app_fv\\RequirementChain.txt"
file_path  = path1 + path2
transaction_execution_commands = neo4j_commands(file_path)
execute_requirementblock(transaction_execution_commands)